## Convolutional AutoEncoder

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import save_image

In [2]:
if not os.path.exists('save/conv_ae'):
    os.mkdir('save/conv_ae')

In [3]:
def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.reshape(x.size(0), 1, 28, 28)
    return x

In [4]:
num_epoches = 100
batch_size = 128
learning_rate = 1e-3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # whether GPU is supportted

In [5]:
img_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [6]:
dataset = datasets.MNIST('../_data/mnist', transform=img_transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [7]:
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, 3, stride=3, padding=1), # b, 16, 10, 10
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=2), # b, 16, 5, 5
            nn.Conv2d(16, 8, 3, stride=2, padding=1), # b, 8, 3, 3
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=1) # b, 8, 2, 2
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(8, 16, 3, stride=2), # b, 16, 5, 5
            nn.ReLU(True),
            nn.ConvTranspose2d(16, 8, 5, stride=3, padding=1), # b, 8, 15, 15
            nn.ReLU(True),
            nn.ConvTranspose2d(8, 1, 2, stride=2, padding=1), # b, 1, 28, 28
            nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [8]:
model = autoencoder().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

In [9]:
for epoch in range(num_epoches):
    for data in dataloader:
        img, _ = data
        img = img.to(device)
        # ===================forward=====================
        output = model(img)
        loss = criterion(output, img)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print(f'epoch [{epoch+1}/{num_epoches}], loss:{loss:.4f}')
    if epoch % 10 == 0:
        pic = to_img(output)
        save_image(pic, 'save/conv_ae/image_{}.png'.format(epoch))

epoch [1/100], loss:0.2225
epoch [2/100], loss:0.1832
epoch [3/100], loss:0.1607
epoch [4/100], loss:0.1493
epoch [5/100], loss:0.1471
epoch [6/100], loss:0.1386
epoch [7/100], loss:0.1285
epoch [8/100], loss:0.1294
epoch [9/100], loss:0.1215
epoch [10/100], loss:0.1158
epoch [11/100], loss:0.1143
epoch [12/100], loss:0.1122
epoch [13/100], loss:0.1156
epoch [14/100], loss:0.1122
epoch [15/100], loss:0.1148
epoch [16/100], loss:0.1105
epoch [17/100], loss:0.1137
epoch [18/100], loss:0.1064
epoch [19/100], loss:0.0952
epoch [20/100], loss:0.1035
epoch [21/100], loss:0.1026
epoch [22/100], loss:0.1071
epoch [23/100], loss:0.1080
epoch [24/100], loss:0.1037
epoch [25/100], loss:0.0972
epoch [26/100], loss:0.0999
epoch [27/100], loss:0.0934
epoch [28/100], loss:0.0948
epoch [29/100], loss:0.0974
epoch [30/100], loss:0.0983
epoch [31/100], loss:0.0927
epoch [32/100], loss:0.0959
epoch [33/100], loss:0.0876
epoch [34/100], loss:0.1022
epoch [35/100], loss:0.0921
epoch [36/100], loss:0.0949
e

In [10]:
torch.save(model.state_dict(), 'save/conv_ae/conv_autoencoder.pytorch')